In [1]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np

In [3]:
df = pd.read_csv("D:/Training/Academy/Gen AI/Cases_AI/hf_dialogsum/dialogue_test.csv")

In [4]:
df.head()

,id,dialogue,summary,topic
0,test_0_1,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson helps #Person1# to write a memo to ...,communication method
1,test_0_2,"#Person1#: Ms. Dawson, I need you to take a di...",In order to prevent employees from wasting tim...,company policy
2,test_0_3,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson takes a dictation for #Person1# abo...,dictation
3,test_1_1,#Person1#: You're finally here! What took so l...,#Person2# arrives late because of traffic jam....,public transportation
4,test_1_2,#Person1#: You're finally here! What took so l...,#Person2# decides to follow #Person1#'s sugges...,transportation


In [5]:
print(df['dialogue'].loc[4])

#Person1#: You're finally here! What took so long?
#Person2#: I got stuck in traffic again. There was a terrible traffic jam near the Carrefour intersection.
#Person1#: It's always rather congested down there during rush hour. Maybe you should try to find a different route to get home.
#Person2#: I don't think it can be avoided, to be honest.
#Person1#: perhaps it would be better if you started taking public transport system to work.
#Person2#: I think it's something that I'll have to consider. The public transport system is pretty good.
#Person1#: It would be better for the environment, too.
#Person2#: I know. I feel bad about how much my car is adding to the pollution problem in this city.
#Person1#: Taking the subway would be a lot less stressful than driving as well.
#Person2#: The only problem is that I'm going to really miss having the freedom that you have with a car.
#Person1#: Well, when it's nicer outside, you can start biking to work. That will give you just as much freedom 

In [7]:
print(df['summary'].loc[4])

#Person2# decides to follow #Person1#'s suggestions on quitting driving to work and will try to use public transportations.


In [21]:
summarizer = pipeline("summarization", model="gsarti/it5-base-news-summarization")

Device set to use cpu


In [22]:
txt_summ = summarizer(df['dialogue'].loc[4])
print(txt_summ[0]['summary_text'])

#person1#: i’m going to driving as well if you started biking at work, it’s not good for me or for the environment, so i do not know how much i have a car in this city.


In [16]:
model_st = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [17]:
embd_true = model_st.encode(df['summary'].loc[4])
embd_infer = model_st.encode(txt_summ[0]['summary_text'])

In [23]:
cosine_similarity([embd_true, embd_infer])

array([[1.        , 0.37610123],
       [0.37610123, 1.0000001 ]], dtype=float32)

In [24]:
def eval_summarize(model_summ, ind):
    txt_summ = model_summ(df['dialogue'].loc[ind])
    embd_true = model_st.encode(df['summary'].loc[ind])
    embd_infer = model_st.encode(txt_summ[0]['summary_text'])
    print("Summary:\n", df['summary'].loc[ind])
    print("Generated Summary:\n", txt_summ[0]['summary_text'])
    print("Similarity: ",cosine_similarity([embd_true, embd_infer])[0,1])
    #return cosine_similarity([embd_true, embd_infer])[0,1]


In [26]:
eval_summarize(summarizer, 56)

Summary:
 #Person2# checks #Person1#'s physical condition and finds #Person1# has a fever.
Generated Summary:
 let me take your temperature with a thermometer. #person1#: i have a small fever? # person2#: what do you think it’s about?
Similarity:  0.6711929


In [ ]:
def eval_summarize(model_summ, ind):
    txt_summ = model_summ(df['dialogue'].loc[ind])
    embd_true = model_st.encode(df['summary'].loc[ind])
    embd_infer = model_st.encode(txt_summ[0]['summary_text'])
    return cosine_similarity([embd_true, embd_infer])[0,1]

In [33]:
sample = [34, 67, 12, 400, 340]
eval_sims = np.array([eval_summarize(summarizer, sample[i]) for i in tqdm(range(len(sample)))])
eval_sims, np.mean(eval_sims)

100%|██████████| 5/5 [00:51<00:00, 10.27s/it]


(array([0.8049084 , 0.6984769 , 0.72592914, 0.90633833, 0.69562745],
       dtype=float32),
 0.766256)